# Grupowanie danych w szeregi czasowe

W skrypcie tym zbiorę dane w szeregi czasowe - ilości poszczególnych danych będę grupował ze względu na ID repozytorium i sumował ich ilość dla każdego miesiąca.  

W przypadku obserwujących, commitów i issues operacje będą bardzo podobne:

- pobranie danych do `DataFrame`
- dołożenie kolumn miesiąca (`month`) i roku (`year`) na podstawie kolumny `created_at`
- pogrupowanie danych ze względu na ID repozytorium, którego dotyczy, miesiąc oraz rok

W przypadku pull requestów, najpierw potrzebne jest dołączenie informacji o historii pull requestów, aby mieć znaczniki czasowe. 

Analogicznie, dla komentarzy commitów i issues potrzebne jest najpierw dołączenie informacji, którego projektu dotyczą.

Dodatkowo po procesie grupowania zmienię nazewnictwo kolumn, aby je ustandaryzować

- ID repozytorium będzie przechowywane w kolumnie `project_id`
- ilość nowych danych będzie przechowywana w kolumnie `new_(nazwa_elementu)`

In [1]:
import pandas as pd

## Liczba obserwujących

In [2]:
watchers = pd.read_pickle('../data/01_data_from_db/watchers.pkl')

In [3]:
watchers

,repo_id,created_at
0,1,2010-05-12 06:16:00
1,1,2009-02-16 12:51:54
2,1,2011-02-09 03:53:14
3,1,2010-09-01 09:05:21
4,1,2009-03-04 09:44:56
...,...,...
295793,107672,2012-06-06 18:26:16
295794,107672,2013-04-15 03:40:11
295795,107672,2012-06-06 18:26:16
295796,107672,2012-07-11 12:33:32


In [4]:
watchers['month'] = watchers['created_at'].dt.month

In [5]:
watchers['year'] = watchers['created_at'].dt.year

In [6]:
watchers.head()

,repo_id,created_at,month,year
0,1,2010-05-12 06:16:00,5,2010
1,1,2009-02-16 12:51:54,2,2009
2,1,2011-02-09 03:53:14,2,2011
3,1,2010-09-01 09:05:21,9,2010
4,1,2009-03-04 09:44:56,3,2009


In [7]:
new_watchers = watchers.groupby(by=['repo_id', 'year', 'month']).count().reset_index().rename(columns={'repo_id': 'project_id', 'created_at': 'new_watchers'})
new_watchers

,project_id,year,month,new_watchers
0,1,2009,2,323
1,1,2009,3,56
2,1,2009,4,29
3,1,2009,5,24
4,1,2009,6,35
...,...,...,...,...
2847,107672,2013,4,6
2848,107672,2013,5,3
2849,107672,2013,6,3
2850,107672,2013,7,6


## Liczba commitów

In [8]:
commits = pd.read_pickle('../data/01_data_from_db/commits.pkl')
commits

,id,project_id,created_at
0,1,10,2012-04-08 02:29:18
1,2,10,2011-12-29 21:12:27
2,3,10,2011-12-29 21:11:10
3,4,10,2011-12-29 20:54:15
4,5,10,2011-12-29 20:29:49
...,...,...,...
255171,262991,78835,2012-12-04 14:17:36
255172,262992,78852,2013-08-28 21:43:33
255173,262993,79166,2013-08-07 15:13:03
255174,262994,79166,2013-08-07 15:12:27


In [9]:
commits['month'] = commits.created_at.dt.month
commits['year'] = commits.created_at.dt.year
commits.head()

,id,project_id,created_at,month,year
0,1,10,2012-04-08 02:29:18,4,2012
1,2,10,2011-12-29 21:12:27,12,2011
2,3,10,2011-12-29 21:11:10,12,2011
3,4,10,2011-12-29 20:54:15,12,2011
4,5,10,2011-12-29 20:29:49,12,2011


In [17]:
new_commits = commits.groupby(['project_id', 'year', 'month']).count().reset_index().rename(columns={'created_at': 'new_commits'}).drop(columns=('id'), axis=1)
new_commits

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


## Liczba issues

In [11]:
issues = pd.read_pickle('../data/01_data_from_db/issues.pkl')
issues

,id,repo_id,created_at
0,1,4,2012-10-10 07:53:00
1,2,4,2012-06-20 15:23:42
2,3,3,2013-05-15 12:48:49
3,4,3,2013-01-29 17:08:05
4,5,3,2013-01-21 17:37:35
...,...,...,...
150357,155021,78852,2013-09-26 19:22:01
150358,155022,78852,2013-09-26 19:02:50
150359,155023,78852,2013-09-26 15:35:11
150360,155024,78852,2013-09-26 14:46:06


In [12]:
issues['month'] = issues.created_at.dt.month
issues['year'] = issues.created_at.dt.year
issues.head()

,id,repo_id,created_at,month,year
0,1,4,2012-10-10 07:53:00,10,2012
1,2,4,2012-06-20 15:23:42,6,2012
2,3,3,2013-05-15 12:48:49,5,2013
3,4,3,2013-01-29 17:08:05,1,2013
4,5,3,2013-01-21 17:37:35,1,2013


In [15]:
new_issues = issues.groupby(['repo_id', 'year', 'month']).count().reset_index().rename(columns={'repo_id': 'project_id', 'created_at': 'new_issues'}).drop(columns='id', axis=1)
new_issues

,project_id,year,month,new_issues
0,1,2009,7,46
1,1,2009,8,9
2,1,2009,9,1
3,1,2009,10,3
4,1,2010,9,6
...,...,...,...,...
2964,107672,2013,6,19
2965,107672,2013,7,15
2966,107672,2013,8,10
2967,107672,2013,9,3


# tmp - eksperymenty z łączeniem zbiorów

In [19]:
new_commits.head()

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1


In [20]:
new_issues.head()

,project_id,year,month,new_issues
0,1,2009,7,46
1,1,2009,8,9
2,1,2009,9,1
3,1,2009,10,3
4,1,2010,9,6


In [22]:
pd.concat([new_commits, new_issues])

,project_id,year,month,new_commits,new_issues
0,2,2010,5,43.0,NaN
1,2,2010,6,4.0,NaN
2,2,2010,7,3.0,NaN
3,2,2010,8,1.0,NaN
4,2,2010,9,1.0,NaN
...,...,...,...,...,...
2964,107672,2013,6,NaN,19.0
2965,107672,2013,7,NaN,15.0
2966,107672,2013,8,NaN,10.0
2967,107672,2013,9,NaN,3.0


In [26]:
new_commits.append(new_issues)

,project_id,year,month,new_commits,new_issues
0,2,2010,5,43.0,NaN
1,2,2010,6,4.0,NaN
2,2,2010,7,3.0,NaN
3,2,2010,8,1.0,NaN
4,2,2010,9,1.0,NaN
...,...,...,...,...,...
21726,107672,2013,6,NaN,19.0
21727,107672,2013,7,NaN,15.0
21728,107672,2013,8,NaN,10.0
21729,107672,2013,9,NaN,3.0


### PRAWDOPODOBNIE POPRAWNE ŁĄCZENIE

przy użyciu metody `merge()`, argumenty `on` dotyczą tego, po jakich kolumnach ma nastąpić łaczenie, `how` oznacza rodzaj (analogicznie do JOIN w SQL), `sort` na `False` może znacznie poprawić wydajność

In [77]:
newbie = pd.merge(new_commits, new_issues, on=['project_id', 'year', 'month'], how='outer', sort=False)
newbie = pd.merge(newbie, new_watchers, on=['project_id', 'year', 'month'], how='outer', sort=False)

In [78]:
newbie.isnull().sum()

project_id          0
year                0
month               0
new_commits      1303
new_issues      17096
new_watchers    17213
dtype: int64

In [80]:
newbie.new_commits.fillna(value=0, inplace=True)
newbie.new_issues.fillna(value=0, inplace=True)
newbie.new_watchers.fillna(value=0, inplace=True)
newbie

,project_id,year,month,new_commits,new_issues,new_watchers
0,2,2010,5,43.0,0.0,182.0
1,2,2010,6,4.0,0.0,8.0
2,2,2010,7,3.0,0.0,13.0
3,2,2010,8,1.0,0.0,10.0
4,2,2010,9,1.0,1.0,22.0
...,...,...,...,...,...,...
20060,107535,2009,11,0.0,0.0,17.0
20061,107535,2009,12,0.0,0.0,26.0
20062,107535,2010,2,0.0,0.0,20.0
20063,107535,2010,7,0.0,0.0,22.0


In [81]:
newbie.dtypes

project_id        int64
year              int64
month             int64
new_commits     float64
new_issues      float64
new_watchers    float64
dtype: object

In [82]:
newbie.new_commits = newbie.new_commits.astype('int64')
newbie.new_issues = newbie.new_issues.astype('int64')
newbie.new_watchers = newbie.new_watchers.astype('int64')
newbie.dtypes

project_id      int64
year            int64
month           int64
new_commits     int64
new_issues      int64
new_watchers    int64
dtype: object

In [83]:
newbie.head()

,project_id,year,month,new_commits,new_issues,new_watchers
0,2,2010,5,43,0,182
1,2,2010,6,4,0,8
2,2,2010,7,3,0,13
3,2,2010,8,1,0,10
4,2,2010,9,1,1,22
